# Image Duplicate Detection using MinHash and LSH


### Objectives:
1. Convert images to feature vectors (using perceptual hashing)
2. Apply **MinHashing** and **Locality Sensitive Hashing (LSH)**
3. Test on a dataset (≥1000 images)
4. Compare results with traditional similarity metrics (e.g., SSIM)


In [52]:
import os
import csv
from pathlib import Path
from PIL import Image
import imagehash
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import cv2
import random

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "images"
RESULTS_DIR = BASE_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

# Parameters
HASH_SIZE = 8          # for perceptual hashing (8 → 64 bits)
NUM_MINHASH = 100      # number of minhash functions
NUM_BANDS = 20         # number of LSH bands (NUM_MINHASH should be divisible by this)
HAMMING_THRESHOLD = 5  # threshold for considering duplicates


In [53]:
def compute_phash(image_path, hash_size=HASH_SIZE):
    """Compute perceptual hash for one image."""
    try:
        img = Image.open(image_path).convert("L")
        ph = imagehash.phash(img, hash_size=hash_size)
        return int(str(ph), 16)
    except Exception as e:
        print(f"Error with {image_path}: {e}")
        return None


hashes_csv = RESULTS_DIR / "hashes.csv"

if not hashes_csv.exists():
    images = sorted([p for p in DATA_DIR.iterdir() if p.suffix.lower() in ('.jpg', '.png', '.jpeg')])
    with open(hashes_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "phash"])
        for p in tqdm(images, desc="Computing perceptual hashes"):
            h = compute_phash(p)
            if h is not None:
                writer.writerow([p.name, h])
    print(f" Hashes saved to {hashes_csv}")
else:
    print("hashes.csv already exists — skipping computation.")


Computing perceptual hashes: 100%|██████████| 1500/1500 [00:17<00:00, 84.72it/s]

 Hashes saved to c:\Users\Ghita\Documents\FALL25\CSC 4352\ImageDuplicatesDetection\Project1\results\hashes.csv


In [54]:
def int_to_bitset(phash_int, n_bits=64):
    """Convert integer pHash to a set of bit positions where bit=1."""
    return {i for i in range(n_bits) if ((phash_int >> i) & 1)}

# Read hashes.csv
rows = []
with open(hashes_csv) as f:
    reader = csv.DictReader(f)
    for r in reader:
        rows.append((r["filename"], int(r["phash"])))

filenames = [r[0] for r in rows]
phashes = [r[1] for r in rows]
bitsets = [int_to_bitset(h) for h in phashes]

print(f"Loaded {len(filenames)} images and built feature sets.")


Loaded 1500 images and built feature sets.


In [55]:
# Generate MinHash functions
def create_hash_funcs(num_hashes=NUM_MINHASH, p=(2**31 - 1)):
    funcs = []
    for _ in range(num_hashes):
        a = random.randint(1, p - 1)
        b = random.randint(0, p - 1)
        funcs.append((a, b, p))
    return funcs


def minhash_signature(feature_set, hash_funcs):
    sig = []
    for a, b, p in hash_funcs:
        sig.append(min(((a * x + b) % p) for x in feature_set))
    return sig


hash_funcs = create_hash_funcs(NUM_MINHASH)
signatures = [minhash_signature(s, hash_funcs) for s in tqdm(bitsets, desc="Computing MinHash signatures")]

print(" MinHash signatures created.")


Computing MinHash signatures: 100%|██████████| 1500/1500 [00:00<00:00, 1982.05it/s]

 MinHash signatures created.


In [56]:
class LSHIndex:
    def __init__(self, num_bands=NUM_BANDS, rows_per_band=None):
        if rows_per_band is None:
            if NUM_MINHASH % num_bands != 0:
                raise ValueError("NUM_MINHASH must be divisible by num_bands")
            rows_per_band = NUM_MINHASH // num_bands
        self.num_bands = num_bands
        self.rows_per_band = rows_per_band
        self.tables = [defaultdict(list) for _ in range(num_bands)]

    def _band_hash(self, band):
        return hash(tuple(band))

    def index(self, doc_id, signature):
        for b in range(self.num_bands):
            start = b * self.rows_per_band
            band = signature[start:start + self.rows_per_band]
            key = self._band_hash(band)
            self.tables[b][key].append(doc_id)

    def get_candidates(self):
        candidates = set()
        for table in self.tables:
            for bucket in table.values():
                if len(bucket) > 1:
                    for i in range(len(bucket)):
                        for j in range(i + 1, len(bucket)):
                            candidates.add(tuple(sorted((bucket[i], bucket[j]))))
        return candidates


lsh = LSHIndex(num_bands=NUM_BANDS)
for idx, sig in enumerate(signatures):
    lsh.index(idx, sig)

candidates = list(lsh.get_candidates())
print(f"Found {len(candidates)} candidate pairs.")


Found 158914 candidate pairs.


In [57]:
def hamming(a, b):
    return bin(a ^ b).count("1")


verified = []
for i, j in candidates:
    fn1, fn2 = filenames[i], filenames[j]
    ph1, ph2 = phashes[i], phashes[j]
    dist = hamming(ph1, ph2)
    if dist <= HAMMING_THRESHOLD:
        verified.append((fn1, fn2, dist))

print(f" Verified {len(verified)} duplicate pairs below threshold {HAMMING_THRESHOLD}.")

# Save results
verified_csv = RESULTS_DIR / "verified_pairs.csv"
with open(verified_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["image1", "image2", "hamming_distance"])
    writer.writerows(verified)

print(f"Results saved to {verified_csv}")


 Verified 949 duplicate pairs below threshold 5.
Results saved to c:\Users\Ghita\Documents\FALL25\CSC 4352\ImageDuplicatesDetection\Project1\results\verified_pairs.csv


In [58]:
def group_duplicates(verified_pairs):
    """
    Group duplicate images using Union-Find algorithm.
    Input: List of (img1, img2, distance) tuples
    Output: List of groups, where each group contains all images that are duplicates of each other
    """
    # Create a mapping from filename to index for easier processing
    all_images = set()
    for img1, img2, _ in verified_pairs:
        all_images.add(img1)
        all_images.add(img2)
    
    image_to_idx = {img: idx for idx, img in enumerate(sorted(all_images))}
    idx_to_image = {idx: img for img, idx in image_to_idx.items()}
    
    # Union-Find data structure
    parent = list(range(len(all_images)))
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])  # Path compression
        return parent[x]
    
    def union(x, y):
        root_x, root_y = find(x), find(y)
        if root_x != root_y:
            parent[root_y] = root_x
    
    # Union all duplicate pairs
    for img1, img2, _ in verified_pairs:
        idx1, idx2 = image_to_idx[img1], image_to_idx[img2]
        union(idx1, idx2)
    
    # Group images by their root parent
    groups = defaultdict(list)
    for idx in range(len(all_images)):
        root = find(idx)
        groups[root].append(idx_to_image[idx])
    
    # Return only groups with more than 1 image, sorted by group size
    duplicate_groups = [sorted(group) for group in groups.values() if len(group) > 1]
    duplicate_groups.sort(key=len, reverse=True)  # Largest groups first
    
    return duplicate_groups

# Group all duplicates
duplicate_groups = group_duplicates(verified)

print(f" Found {len(duplicate_groups)} duplicate groups:")
print(f" Total images involved in duplicates: {sum(len(group) for group in duplicate_groups)}")

# Display group statistics
group_sizes = defaultdict(int)
for group in duplicate_groups:
    group_sizes[len(group)] += 1

print(f"\n Group size distribution:")
for size in sorted(group_sizes.keys(), reverse=True):
    count = group_sizes[size]
    total_images = size * count
    print(f"   Groups of {size} images: {count} groups ({total_images} total images)")

# Show the largest groups
print(f"\n Top 10 largest duplicate groups:")
for i, group in enumerate(duplicate_groups[:10], 1):
    print(f"   Group {i} ({len(group)} images): {', '.join(group[:5])}{'...' if len(group) > 5 else ''}")

 Found 523 duplicate groups:
 Total images involved in duplicates: 1218

 Group size distribution:
   Groups of 4 images: 82 groups (328 total images)
   Groups of 3 images: 8 groups (24 total images)
   Groups of 2 images: 433 groups (866 total images)

 Top 10 largest duplicate groups:
   Group 1 (4 images): image1001.jpg, image1456.jpg, image1495.jpg, image756.jpg
   Group 2 (4 images): image1008.jpg, image350.jpg, image61.jpg, image766.jpg
   Group 3 (4 images): image101.jpg, image216.jpg, image483.jpg, image704.jpg
   Group 4 (4 images): image1010.jpg, image219.jpg, image463.jpg, image664.jpg
   Group 5 (4 images): image1024.jpg, image377.jpg, image528.jpg, image898.jpg
   Group 6 (4 images): image1027.jpg, image1434.jpg, image360.jpg, image404.jpg
   Group 7 (4 images): image1029.jpg, image1395.jpg, image160.jpg, image847.jpg
   Group 8 (4 images): image103.jpg, image1292.jpg, image1455.jpg, image425.jpg
   Group 9 (4 images): image1032.jpg, image1152.jpg, image25.jpg, image976.j

In [59]:
# Save grouped results to CSV
grouped_csv = RESULTS_DIR / "duplicate_groups.csv"

try:
    with open(grouped_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["group_id", "group_size", "images"])
        
        for i, group in enumerate(duplicate_groups, 1):
            writer.writerow([f"Group_{i}", len(group), "; ".join(group)])
    print(f" Duplicate groups saved to: {grouped_csv}")
except PermissionError:
    print(f" Error: Cannot write to {grouped_csv}")
    print("   The file is currently open in another program (Excel, text editor, etc.)")
    print("   Please close the file and run this cell again.")

# Also create a detailed pairs file that shows the relationships
detailed_csv = RESULTS_DIR / "detailed_duplicate_pairs.csv"

try:
    with open(detailed_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["group_id", "image1", "image2", "hamming_distance"])
        
        # Map each image to its group
        image_to_group = {}
        for group_idx, group in enumerate(duplicate_groups, 1):
            for image in group:
                image_to_group[image] = f"Group_{group_idx}"
        
        # Add group information to verified pairs
        for img1, img2, dist in verified:
            group_id = image_to_group.get(img1, "Unknown")
            writer.writerow([group_id, img1, img2, dist])
    print(f" Detailed pairs with groups saved to: {detailed_csv}")
except PermissionError:
    print(f" Error: Cannot write to {detailed_csv}")
    print("   The file is currently open in another program.")
    print("   Please close the file and run this cell again.")

# Summary statistics
total_images_in_dataset = len(filenames)
images_with_duplicates = sum(len(group) for group in duplicate_groups)
unique_images = total_images_in_dataset - images_with_duplicates + len(duplicate_groups)

print(f"\n Dataset Summary:")
print(f"   Total images in dataset: {total_images_in_dataset}")
print(f"   Images with duplicates: {images_with_duplicates}")
print(f"   Unique images (no duplicates): {unique_images}")
print(f"   Duplicate rate: {(images_with_duplicates/total_images_in_dataset)*100:.2f}%")

 Duplicate groups saved to: c:\Users\Ghita\Documents\FALL25\CSC 4352\ImageDuplicatesDetection\Project1\results\duplicate_groups.csv
 Detailed pairs with groups saved to: c:\Users\Ghita\Documents\FALL25\CSC 4352\ImageDuplicatesDetection\Project1\results\detailed_duplicate_pairs.csv

 Dataset Summary:
   Total images in dataset: 1500
   Images with duplicates: 1218
   Unique images (no duplicates): 805
   Duplicate rate: 81.20%


In [60]:
def compute_ssim(path1, path2):
    a = cv2.imread(str(path1), cv2.IMREAD_GRAYSCALE)
    b = cv2.imread(str(path2), cv2.IMREAD_GRAYSCALE)
    if a is None or b is None:
        return None
    h, w = min(a.shape[0], b.shape[0]), min(a.shape[1], b.shape[1])
    a = cv2.resize(a, (w, h))
    b = cv2.resize(b, (w, h))
    return ssim(a, b)

sample = verified[:30]
ssim_scores = []
for fn1, fn2, dist in sample:
    s = compute_ssim(DATA_DIR / fn1, DATA_DIR / fn2)
    ssim_scores.append((fn1, fn2, dist, s))

import pandas as pd
df = pd.DataFrame(ssim_scores, columns=["Image1", "Image2", "Hamming", "SSIM"])
df.head(10)


,Image1,Image2,Hamming,SSIM
0,image845.jpg,image986.jpg,0,1.0
1,image393.jpg,image843.jpg,0,1.0
2,image1401.jpg,image1450.jpg,0,1.0
3,image1299.jpg,image897.jpg,0,1.0
4,image417.jpg,image505.jpg,0,1.0
5,image207.jpg,image302.jpg,0,1.0
6,image1296.jpg,image557.jpg,0,1.0
7,image1310.jpg,image876.jpg,0,1.0
8,image395.jpg,image988.jpg,0,1.0
9,image101.jpg,image216.jpg,0,1.0


### Pipeline Workflow

**Step 1: Feature Extraction (Perceptual Hashing)**
- Each image is converted into a perceptual hash (pHash) — a 64-bit compact representation
- pHash captures the visual "essence" of an image while being robust to minor variations
- Unlike cryptographic hashes, perceptual hashes produce similar values for visually similar images
- The hash is computed from the Discrete Cosine Transform (DCT) of the grayscale image

**Step 2: Set Representation**
- Each 64-bit hash is converted into a set of bit positions where the bit value is 1
- This set representation enables the use of Jaccard similarity and MinHash techniques
- Example: hash `0b1010...` becomes set `{0, 2, ...}` (positions of 1-bits)

**Step 3: MinHash Signature Generation**
- Apply 100 hash functions to each bit-position set to create MinHash signatures
- MinHash preserves Jaccard similarity: similar sets produce similar signatures
- Reduces comparison complexity while maintaining similarity relationships
- Each signature is a 100-element vector of minimum hash values

**Step 4: Locality Sensitive Hashing (LSH)**
- Divide each 100-element signature into 20 bands of 5 rows each
- Hash each band independently and group images with matching band hashes into buckets
- Images in the same bucket are candidate duplicates (probability increases with more matching bands)
- This reduces the search space from O(n²) to approximately O(n) for finding similar pairs

**Step 5: Verification**
- Candidate pairs are verified by computing Hamming distance between original pHashes
- Only pairs with Hamming distance ≤ 5 bits (out of 64) are confirmed as duplicates
- This threshold allows ~7.8% difference, tolerating minor variations
- Additionally, SSIM (Structural Similarity Index) is computed for comparison with traditional methods

**Step 6: Grouping**
- Verified duplicate pairs are grouped using Union-Find (disjoint set) algorithm
- This identifies clusters where A≈B and B≈C implies all three are duplicates
- Results in transitive duplicate groups rather than just pairwise matches

### Comparison: MinHash/LSH vs Traditional Methods (SSIM)

| Aspect | Perceptual Hash + MinHash + LSH | SSIM (Traditional) |
|--------|----------------------------------|---------------------|
| **Type** | Probabilistic, hash-based | Deterministic, pixel-based |
| **Scalability** |O(n) with LSH | O(n²) comparisons needed |
| **Speed** | Very fast (milliseconds per image) | Slow (requires full pixel comparison) |
| **Memory** | Low (64-bit hash per image) | High (needs full images loaded) |
| **Near-duplicate detection** | Tolerates minor changes | Very sensitive to small pixel changes |
| **Exact duplicate detection** | Good | Good |
| **Rotation invariance** | Not built-in (needs modification) | Not rotation-invariant |
| **Resize/crop tolerance** | Limited tolerance | Requires same dimensions |
| **Heavy transformations** | Misses heavily edited images | Also misses major transformations |
| **Best use case** | Large-scale duplicate detection (millions of images) | Small-scale, high-accuracy verification |

### Key Findings:

**Strengths of MinHash + LSH:**
- Scales to massive datasets (tested on 1000+ images here, works for millions)
- Fast candidate generation via LSH banding (avoids all-pairs comparison)
- Effective for finding near-duplicates with minor variations (compression artifacts, slight color adjustments, minor crops)
- Memory efficient with compact hash representations
- Adjustable sensitivity via `HAMMING_THRESHOLD` and LSH band configuration

**Limitations:**
- Requires careful threshold tuning (`HAMMING_THRESHOLD`, `NUM_BANDS`)
- May miss images with significant transformations (heavy filters, rotations, major crops)
- Probabilistic nature means some similar pairs might be missed (false negatives)
- Can produce false positives if threshold is too high
- Not robust to rotation without additional preprocessing

**When to Use SSIM:**
- Final verification step (as demonstrated in this notebook)
- Small datasets where O(n²) comparisons are computationally feasible
- Need precise pixel-level similarity scores (0.0 to 1.0 range)
- Quality assessment for image processing pipelines
- Ground truth validation for other methods

**Performance Considerations:**
- **LSH Tuning**: With 20 bands × 5 rows, probability of detection ≈ (1-(1-s^5)^20) where s is Jaccard similarity
- **Threshold Selection**: `HAMMING_THRESHOLD = 5` allows ~7.8% bit differences, suitable for compressed/resized duplicates
- **Trade-off**: More bands → higher precision but lower recall; fewer bands → higher recall but more false positives

**Conclusion:**  
The MinHash + LSH approach successfully detects duplicate and near-duplicate images efficiently at scale, making it suitable for real-world applications like photo deduplication, copyright detection, and content moderation. SSIM serves as an excellent validation metric but is impractical as a primary detection method for large datasets due to its O(n²) complexity. This hybrid approach leverages the speed of LSH for candidate generation and the accuracy of traditional metrics for verification, achieving both scalability and reliability.